In [1]:
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import gensim
import pandas as pd
import collections
import sys

## Train doc2vec
- using DBOW with dm_concat=1 give better result than using DM with dm_concat=1
- DM, concat: 
    + word vector:
    + sentence vector:
    
- DBOW, concat: 
    + word vector: bad
    + sentence vector:

- DM, sum: 
    + word vector: ok
    + sentence vector: average
    
- DBOW, sum: 
    + word vector: bad
    + sentence vector: good

- DM, mean: 
    + word vector: ok
    + sentence vector: average
    
- DBOW, mean:
    + word vector: bad
    + sentence vector: good?


In [4]:
 models = {
        'dm_concat': None,
#         'dbow_concat': None,
#         'dm_sum': None,
#         'dbow_sum': None,
#         'dm_mean': None,
#         'dbow_mean': None
    }
pattern = '../model/doc2vec/doc2vec_external_%s.vec'

In [8]:
# load all documents
def iterdocuments(filenames, tokens_only=False):
    """
    Iterate over documents, yielding a list of utf8 tokens at a time.

    Args:
        filenames (TYPE): Description
        encoding (str, optional): Description

    Yields:
        TYPE: Description
    """
    index = 0
    for filename in filenames:
        df = pd.read_csv(filename, names=['text'])
        for _, row in df.iterrows():
            if tokens_only:
                yield gensim.utils.simple_preprocess(row['text'])
            else:
                yield gensim.models.doc2vec.TaggedDocument(
                    gensim.utils.simple_preprocess(row['text']), [index])
                index += 1

train_corpus = list(iterdocuments(['../data/sentences_data.csv']))
len(train_corpus)

361003

In [ ]:
for k in models.keys():
    models[k] = pattern % k
    model = gensim.models.doc2vec.Doc2Vec.load(models[k])
    print('\nMODEL:', k)
    print('Number of documents:', len(model.docvecs))
    # get similarity by words
    compared_words = ['ram', 'bộ_nhớ', 'dung_lượng']
    for w in compared_words:
        print('Compare with "%s"' % w.upper())
        similars = model.wv.similar_by_word(w, topn=10)
        print('\t' + ' '.join(['(%s, %.2f)' % (x[0], x[1]) for x in similars]))
    
    # get similar document
    compared_document_ids = [69925, 69936, 83035]
    for doc_id in compared_document_ids:
        inferred_vector = model.infer_vector(train_corpus[doc_id].words)
        sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
        print('\nDocument ({}): «{}»'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
        print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
        # for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
        #     print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))
        for index in range(20):
            print(u'%s: «%s»' % (sims[index], ' '.join(train_corpus[sims[index][0]].words)))

In [15]:
np.random.randint(1, high=len(train_corpus), size=3)

array([230804,  86509, 184852])

In [13]:
# model.wv.similar_by_word('bộ_nhớ', topn=10)

In [14]:
# model.wv.similar_by_word('dung_lượng', topn=10)

In [54]:
inferred_vector = model.infer_vector(['điện_thoại'])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
for index in range(20):
    print(u'%s: «%s»' % (sims[index], ' '.join(train_corpus[sims[index][0]].words)))

(176697, 0.8649290800094604): «điện_thoại»
(107748, 0.8645000457763672): «điện_thoại»
(243107, 0.8614373803138733): «điện_thoại»
(249427, 0.852385401725769): «điện_thoại wiko_sunny»
(16070, 0.844206690788269): «điện_thoại»
(243186, 0.8438059687614441): «điện_thoại»
(245230, 0.8383656144142151): «điện_thoại masstel_fami»
(244177, 0.837456226348877): «điện_thoại wiko_robby»
(19986, 0.8273682594299316): «điện_thoại»
(245338, 0.8181977272033691): «điện_thoại ivvi»
(244220, 0.8088253140449524): «điện_thoại»
(11866, 0.7841490507125854): «điện_thoại»
(138124, 0.7770631909370422): «số điện_thoại»
(60053, 0.7766164541244507): «số điện_thoại»
(110019, 0.7734436392784119): «điện_thoại nhé»
(8687, 0.7610524892807007): «google»
(204653, 0.7505836486816406): «số điện_thoại»
(79766, 0.7457122802734375): «tờ hi_chào bạn»
(93906, 0.7446766495704651): «số điện_thoại»
(145877, 0.7365298271179199): «điện_thoại của mình mua từ»


In [56]:
doc_id = 251632
inferred_vector = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
# for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
#     print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))
for index in range(30):
    print(u'%s: «%s»' % (sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (251632): «bên cạnh đó bộ_nhớ trong với dung_lượng gb sẽ đáp_ứng tốt nhu_cầu lưu_trữ dữ_liệu của bạn»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d400,n5,hs,mc5,s0.0001,t3):

(251632, 0.7037237882614136): «bên cạnh đó bộ_nhớ trong với dung_lượng gb sẽ đáp_ứng tốt nhu_cầu lưu_trữ dữ_liệu của bạn»
(243956, 0.5447922945022583): «bên cạnh đó cool dual còn được trang_bị bộ_nhớ trong dung_lượng lên đến gb cho phép người dùng thoải_mái lưu_trữ những dữ_liệu cần_thiết»
(246367, 0.5384358167648315): «máy cũng hỗ_trợ thẻ_nhớ dung_lượng gb giúp bạn mở_rộng bộ_nhớ để lưu_trữ dữ_liệu hình_ảnh»
(247378, 0.5149009227752686): «thêm vào đó bộ_nhớ trong gb cho bạn thỏa_sức lưu_trữ dữ_liệu hình_ảnh»
(244982, 0.5140997171401978): «bộ_nhớ trong gb giúp người dùng thoải_mái lưu_trữ dữ_liệu phục_vụ giải_trí và công_việc ngoài_ra máy còn trang_bị khe cắm thẻ_nhớ microsd hỗ_trợ lên tới gb đáp_ứng các nhu_cầu cao hơn của người dùng»
(244754, 0.5128475427627563): «bên cạnh đó máy có bộ_nhớ ram gb cù

In [ ]:
# Assessing Model
ranks = []
second_ranks = []
total = len(train_corpus)
max_length = 30
pattern = "[%-30s] %d%%"
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])
    sys.stdout.write('\r'
    # the exact output you're looking for
    frac = int((doc_id + 1) * 100. / total)
    cur_length = int(frac * max_length * 1. / 100)
    sys.stdout.write(pattern % ('=' * cur_length, frac))
    sys.stdout.flush()

In [ ]:
collections.Counter(ranks)

In [ ]:
# convert documents to vector
def iterdocuments_to_vector(filenames, vectorizer_model):
    """
    Iterate over documents, yielding vector at a time.

    Args:
        filenames (TYPE): Description
        
    Yields:
        TYPE: Description
    """
    for filename in filenames:
        df = pd.read_csv(filename, header=None, names=['text'])
        logger.info('corpus: %s, size: %s',
                    os.path.basename(filename),
                    df.shape[0])
        for _, row in df.iterrows():
            yield vectorizer_model.infer_vector(row['text'].split())

## Cluster documents of Organizer

In [ ]:
# the number of document is too large, we cannot use Affinity Propagation
# option 1: use Kmean
# option 2: use hierarchy clustering

In [ ]:
X = np.array([model.docvecs[i] for i in range(1000)])

In [ ]:
# option 1: use Kmean
kmeans_model = KMeans(n_clusters=300, random_state=1).fit(X)
labels = kmeans_model.labels_
metrics.calinski_harabaz_score(X, labels)

In [ ]:
# option 2: use hierarchy clustering
